In [1]:
import numpy as np
from scipy.linalg import expm
import pennylane as qml
from pennylane import X, Y, Z, I
from kak_tools import identify_algebra, split_pauli_algebra, map_simple_to_irrep, map_irrep_to_matrices

In [2]:
def close_and_split(gens, verbose=True):
    gens = [list(op.pauli_rep)[0] for op in gens] # Map generators from Operator to PauliWord
    dla = qml.lie_closure(gens, pauli=True)
    dla = [list(op)[0] for op in dla] # Map dla from PauliSentence to PauliWord
    np.random.shuffle(dla)
    sub_dlas = split_pauli_algebra(dla, verbose=verbose)
    return dla, sub_dlas

In [83]:
# Test with so(2n)
n = 8
m = 7
gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
gens += [X(w) @ Y(w+1) for w in range(n, m+n-1)]
dla, sub_dlas = close_and_split(gens)
# print(sub_dlas[0])
# print(sub_dlas[1])
identifiers = identify_algebra(sub_dlas, verbose=True)
print(identifiers)

Found 2 components with dimensions [120, 21].
Dimension of component: 120.
Component 0 can be one of the following:
so(16)
8 copies of so(6)
Dimension of component: 21.
Component 1 can be one of the following:
so(7)
sp(3)
[[(1, 'so', 16), (8, 'so', 6)], [(1, 'so', 7), (1, 'sp', 3)]]


In [84]:
# # Test with random generators
# def make_random_semisimple_algebra(num_wires=5, num_gens=7, seed=None):
#     rng = np.random.default_rng(seed=seed)
#     failure = True
#     while failure:
#         words = rng.choice([I, X, Y, Z], replace=True, size=(num_gens, num_wires))
#         gens = [qml.prod(*(P(w) for w, P in enumerate(word))) for word in words]
#         dla, sub_dlas = close_and_split(gens, verbose=False)
#         dims = [len(sub) for sub in sub_dlas]
#         if 1 not in dims:
#             failure = False
#     return dla
    
# num_wires = 8
# num_gens = 10
# seed = 2
# gens = make_random_semisimple_algebra(num_wires, num_gens, seed=seed)
# # print(f"Max DLA dimension: {4**num_wires-1}")
# dla, sub_dlas = close_and_split(gens)
# # print(sub_dlas)
# # identifiers = identify_algebra(sub_dlas, verbose=True)

## Map to irrep and associated matrices

In [100]:
import numpy as np
from itertools import combinations, product
from pennylane import X, Y, Z, I, lie_closure
from kak_tools import map_simple_to_irrep, map_irrep_to_matrices, identify_algebra, split_pauli_algebra, bdi, lie_closure_pauli_words

n = 32 # Number of qubits
n_so = 2 * n # The "n" in so(n)
sub_hor_size = 3 * n # Number of random generators that we demand to be mapped to horizontal space (Hamiltonian terms in app)

so_dim = (n_so**2-n_so) // 2 # dimension of so(n_so) = so(2n)
print(f"{n=}; so(2n) dim: {so_dim}")

gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)] # Generators as in FDHS paper
# gens = [Y(w) @ X(w+1) for w in range(n-1)] + [X(w) @ Y(w+1) for w in range(n-1)] + [Y(w) for w in range(n)]
gens = [next(iter(op.pauli_rep)) for op in gens] # Map from qml.operation.Operator to qml.pauli.PauliWord


dla = lie_closure_pauli_words(gens, verbose=True, full_size=so_dim) # Compute Lie closure
print("Closed DLA")

n=32; so(2n) dim: 2016
epoch 1 of lie_closure, DLA size is 94
epoch 2 of lie_closure, DLA size is 216
epoch 3 of lie_closure, DLA size is 448
epoch 4 of lie_closure, DLA size is 864
epoch 5 of lie_closure, DLA size is 1504
After 5 epochs, reached a DLA size of 2016
Closed DLA


In [101]:
# dla = [next(iter(op)) for op in dla] # The Lie closure returns PauliSentences, but we want PauliWord's again.
assert len(dla) == so_dim, f"{len(dla)}, {so_dim}" # Assert that we indeed got so(n_so) = so(2n)

invol_type = "BDI"

if invol_type == "BDI":
    def theta(pw):
        """Concurrence ("number of Ys") involution to be applied to PauliWord instances."""
        return -(-1) ** sum(p=="Y" for p in pw.values())

    hor_dim = (n_so//2) ** 2

elif invol_type == "DIII":

    def theta(pw):
        """Even-Odd involution to be applied to PauliWord instances."""
        # return -(-1) ** sum(p=="X" for p in pw.values())
        return -(-1) ** (pw.commutes_with(qml.pauli.PauliWord({i: "Y" for i in pw})))

    hor_dim = (n_so//2) ** 2 - n_so // 2

# for pw in dla:
#     print(pw, theta(pw))

# k = [pw for pw in dla if theta(pw) == 1]
# m = [pw for pw in dla if theta(pw) == -1]
# for pw1, pw2 in combinations(k, r=2):
#     if pw1.commutes_with(pw2):
#         continue
#     assert (com:=pw1._commutator(pw2)[0]) in k, f"{pw1} | {pw2} => {com}"

# for pw1, pw2 in product(k, m):
#     if pw1.commutes_with(pw2):
#         continue
#     assert (com:=pw1._commutator(pw2)[0]) in m, f"{pw1} | {pw2} => {com}"

# for pw1, pw2 in combinations(m, r=2):
#     if pw1.commutes_with(pw2):
#         continue
#     assert (com:=pw1._commutator(pw2)[0]) in k, f"{pw1} | {pw2} => {com}"

# assert len(k) == len(lie_closure(k, pauli=True))
# print(identify_algebra(dla, verbose=True))
# comps = split_pauli_algebra(k)
# print(*comps, sep="\n")
# print(f"k={identify_algebra(comps, verbose=True)}")
# print(len(k))
# print(len(m))
# print(len(dla))

# Make sure we're not trying to squeeze too many operators into the horizontal space (consistency check)
print(hor_dim)
if sub_hor_size > hor_dim:
    raise ValueError("Not enough room in the horizontal space")

# Generate random horizontal Pauli words
hor_gens = gens
sub_hor_size = len(gens)
hor_gens = np.random.choice([op for op in dla if theta(op) == -1], replace=False, size=sub_hor_size)

1024


In [102]:
# Map index pairs corresponding to E_{ij} matrices to operators in the DLA
mapping, signs = map_simple_to_irrep(dla, hor_gens, n=n_so, invol_type=invol_type)
# ex_op = mapping[(0, 4)]
# print(f"E_{{0, 4}} is mapped to {ex_op}")
# Get rid of the index pairs and map operators in the DLA to matrices directly
matrix_map = map_irrep_to_matrices(mapping, signs, n_so, invol_type=invol_type)

In [103]:
H_coeffs = np.random.normal(0, 1., sub_hor_size) # Make up some Hamiltonian coefficients
H = np.tensordot(H_coeffs, np.stack([matrix_map[op] for op in hor_gens]), axes=[[0], [0]]) # Compute Hamiltonian matrix

In [104]:
time = 0.6
U = expm(-time * H)

def embed(op, start, end, n):
    mat = np.zeros((n, n), dtype=complex)
    mat[:start, :start] = np.eye(start)
    mat[start:end, start:end] = op
    mat[end:, end:] = np.eye(n-end)
    return mat

def recursive_bdi(U, n, num_iter=1):
    p = q = n // 2
    k1, a, k2 = bdi(U, p, q)
    ops = {-1: [(U, 0, n, None)], 0: [(k1, 0, n, "k"), (a, 0, n, "a"), (k2, 0, n, "k")]}
    for i in range(1, num_iter+1):
        print(i)
        p = q = p // 2
        print(p, q)
        new_ops = []
        for k, (op, start, end, _type) in enumerate(ops[i-1]):
            # print(np.round(op,1))
            if _type == "a":
                # CSA element
                new_ops.append((op, start, end, "a"))
                continue
            width = end - start
            assert width % 2 == 0
            for s, e in [(start, start+width//2), (start+width//2, end)]:
                # print(p, q, s, e, op.shape)
                k1, a, k2 = bdi(op[s:e, s:e], p, q, is_horizontal=False)
                # print(f"With {p=} and {q=}, decomposed {op[s:e, s:e]} into\n{k1}\n{a}\n{k2}")
                k1, a, k2 = embed(k1, s, e, n), embed(a, s, e, n), embed(k2, s, e, n)
                new_ops.extend([(k1, s, e, "k"), (a, s, e, "a"), (k2, s, e, "k")])
        ops[i] = new_ops
        # ops[(i, 1)] = bdi(ops[(0, 0)][0][2*p:, 2*p:], p, q, is_horizontal=False)
        # ops[(i, 2)] = (ops[(1, 1)][2].T, ops[(1, 1)][1].T, ops[(1, 1)][0].T)
        # ops[(i, 3)] = (ops[(1, 0)][2].T, ops[(1, 0)][1].T, ops[(1, 0)][0].T)
        
        # rec_k1 = np.block([[ops[(1, 0)][0] @ ops[(1, 0)][1] @ ops[(1, 0)][2], np.zeros(ops[(1, 0)][2].shape)], [np.zeros(ops[(1, 0)][2].shape), ops[(1, 1)][0] @ ops[(1, 1)][1] @ ops[(1, 1)][2]]])
        # rec_k2 = np.block([[ops[(1, 3)][0] @ ops[(1, 3)][1] @ ops[(1, 3)][2], np.zeros(ops[(1, 2)][2].shape)], [np.zeros(ops[(1, 2)][2].shape), ops[(1, 2)][0] @ ops[(1, 2)][1] @ ops[(1, 2)][2]]])
        # rec = rec_k1 @ ops[(0, 0)][1] @ rec_k2
        # np.allclose(rec, U)
    return ops, width // 4


num_iter = 5
ops, smallest_block_size = recursive_bdi(U, n_so, num_iter=num_iter)
print(f"Decomposed into {len(ops[num_iter])} blocks with (partially large) CSA blocks and K blocks of size {smallest_block_size}")

def num_ops(_iter):
    if _iter==0:
        return 3
    return num_ops(_iter - 1) + 4 ** _iter * 5 // 2

# Validate
for i in range(num_iter+1):
    print(f"{i=}")
    print(f"{len(ops[i])}, {num_ops(i)}")
    assert len(ops[i]) == num_ops(i), f"{len(ops[i])}, {num_ops(i)}"
    for tup in ops[i]:
        assert isinstance(tup, tuple) and len(tup) == 4
        assert isinstance(tup[0], np.ndarray) and tup[0].shape == (n_so, n_so)
        for val in tup[1:3]:
            assert isinstance(val, int) and 0<=val<=n_so
        assert tup[3] in "ak"

    pos = 0
    if i > 0:
        for j in range(len(ops[i-1])):
            if ops[i-1][j][3] == "k":
                rec = ops[i][pos][0] @ ops[i][pos+1][0] @ ops[i][pos+2][0] @ ops[i][pos+3][0] @ ops[i][pos+4][0] @ ops[i][pos+5][0]
                assert np.allclose(rec, ops[i-1][j][0])
                pos += 6
            else:
                assert ops[i-1][j][3] == "a"
                assert np.allclose(ops[i][pos][0], ops[i-1][j][0])
                pos += 1

1
16 16
2
8 8
3
4 4
4
2 2
5
1 1
Decomposed into 3413 blocks with (partially large) CSA blocks and K blocks of size 1
i=0
3, 3
i=1
13, 13
i=2
53, 53
i=3
213, 213
i=4
853, 853
i=5
3413, 3413


In [90]:
# from pennylane.labs.dla import structure_constants_dense

# all_mats = -1j * np.stack([matrix_map[pw] for pw in dla])
# print(all_mats.shape)
# for i in range(len(all_mats)):
#     if not np.allclose(all_mats[i].conj().T, all_mats[i]):
#         print(all_mats[i])
# adj_pw = qml.structure_constants(dla)
# adj_mat = structure_constants_dense(all_mats) * n_so
# print(np.allclose(adj_pw, adj_mat))
# print(np.allclose(np.abs(adj_pw), np.abs(adj_mat)))
# for i in range(len(adj_mat)):
#     if not np.allclose(adj_mat[i], adj_pw[i]):
#         ids = np.where(~(adj_mat[i]==adj_pw[i]))
#         print(i, ids)
#         # print(adj_mat[i][ids])
#         # print(adj_pw[i][ids])
#         # print()
#         # break